In [82]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / "src"))

In [84]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from copy import deepcopy
import seaborn.objects as so
import plotly.express as px
import pandas as pd


from variables import get_db_path
from db_commands.connect import connect_to_real_db 
from db_commands.read import read_all_entries, aggregate_focus_project_date


In [107]:
connection = connect_to_real_db(get_db_path())
df = read_all_entries(connection)
df_agg = aggregate_focus_project_date(df)
df_agg = df_agg.group_by("focus_area", "date").agg([pl.col("elapsed_time").sum(), pl.col("project")])
df_agg

focus_area,date,elapsed_time,project
str,str,duration[ms],list[str]
"""phd""","""2024-12-13""",32m,"[""SERC""]"
"""personal_code""","""2024-12-18""",1h 47m,"[""slowcial""]"
"""foundations""","""2024-12-18""",1h 30m,"[""probabilistic graphical models""]"
"""personal_code""","""2024-12-05""",2h 19m,"[""time_logger""]"
"""career""","""2024-12-20""",2h 37m,"[""summer24 job application""]"
"""career""","""2024-12-13""",3h 39m,"[""summer24 job application""]"
"""admin""","""2024-12-14""",3h 17m,"[""cee243""]"
"""personal_code""","""2024-12-07""",1h 55m,"[""slowcial""]"
"""personal_code""","""2024-12-11""",1h 49m,"[""time_logger""]"


In [108]:
df_pandas = df_agg.with_columns(elapsed_time=pl.col("elapsed_time").cast(datetime.timedelta)).to_pandas()
df_pandas["elapsed_time"] = df_pandas["elapsed_time"].dt.to_pytimedelta()
df_pandas

,focus_area,date,elapsed_time,project
0,phd,2024-12-13,0 days 00:32:00,[SERC]
1,personal_code,2024-12-18,0 days 01:47:00,[slowcial]
2,foundations,2024-12-18,0 days 01:30:00,[probabilistic graphical models]
3,personal_code,2024-12-05,0 days 02:19:00,[time_logger]
4,career,2024-12-20,0 days 02:37:00,[summer24 job application]
5,career,2024-12-13,0 days 03:39:00,[summer24 job application]
6,admin,2024-12-14,0 days 03:17:00,[cee243]
7,personal_code,2024-12-07,0 days 01:55:00,[slowcial]
8,personal_code,2024-12-11,0 days 01:49:00,[time_logger]
9,career,2024-12-06,0 days 01:32:00,[building africa's cities summit]


In [124]:
df_time = df_agg.group_by("date").agg(pl.col("elapsed_time").sum()).to_pandas()
df_time

,date,elapsed_time
0,2024-12-20,0 days 02:37:00
1,2024-12-11,0 days 01:49:00
2,2024-12-07,0 days 01:55:00
3,2024-12-18,0 days 03:17:00
4,2024-12-13,0 days 04:11:00
5,2024-12-14,0 days 03:17:00
6,2024-12-05,0 days 02:19:00
7,2024-12-06,0 days 01:32:00


In [125]:
min_time = df_time["elapsed_time"].min()
max_time = df_time["elapsed_time"].max()

# TODO fx, where increase the frequency if the number of tick vals
tick_values = pd.timedelta_range("0", max_time, freq="30min", )
tick_ints = tick_values.astype(int)
format_timedelta = lambda v: f"{v.components.hours:0>2d}:{v.components.minutes:0>2d}"
tick_strings = [format_timedelta(i) for i in tick_values]
df_pandas["Time"] = df_pandas["elapsed_time"].apply(format_timedelta)
tick_strings

['00:00',
 '00:30',
 '01:00',
 '01:30',
 '02:00',
 '02:30',
 '03:00',
 '03:30',
 '04:00']

In [126]:
fig = px.bar(df_pandas, x='date', y='elapsed_time', color="focus_area", hover_name="focus_area", hover_data={"date": True, "focus_area": False, "Time": True, "elapsed_time": False, "project": True})
fig.update_layout(yaxis={
        "tickmode": "array",
        "tickvals": tick_ints,
        "ticktext": tick_strings
})


fig.show()

In [ ]:
df_agg2 = aggregate_focus_project_date(df)


focus_area,project,date,elapsed_time
str,str,str,duration[ms]
"""phd""","""SERC""","""2024-12-13""",32m
"""personal_code""","""slowcial""","""2024-12-18""",1h 47m
"""career""","""summer24 job application""","""2024-12-13""",3h 39m
"""personal_code""","""slowcial""","""2024-12-07""",1h 55m
"""personal_code""","""time_logger""","""2024-12-11""",1h 49m
"""career""","""summer24 job application""","""2024-12-20""",2h 37m
"""foundations""","""probabilistic graphical models""","""2024-12-18""",1h 30m
"""personal_code""","""time_logger""","""2024-12-05""",2h 19m
"""career""","""building africa's cities summi…","""2024-12-06""",1h 32m
